In [2]:
#| label: setup
#| echo: false

import difflib as dl
import re
from pathlib import Path

import duckdb
import numpy as np
import pandas as pd
#import plotly.express as px
#from itables import init_notebook_mode  # displays dataframes in friendly manner

#init_notebook_mode(
#    all_interactive=True
#)

In [3]:
#| label: data_source
#| echo: false

DATA_URL = "https://hbiostat.org/data/repo/titanic3.csv"  # is this "definitive"

## Did a male octogenarian really survive the sinking of the RMS Titanic?

### Or: Is there a long-standing error in an oft-used dataset?

As it’s not necessarily a word we use often, let me paraphrase: did an 80 year old guy really manage to make it out of the freezing waters to safety following the infamous maritime disaster?

The short answer is NO. However, read on and let me explain how this article came to be as part of my Data Science travels – in a [Kaggle](http://www.kaggle.com) warm-up “competition” specifically.

### Source data - "1999 Original" (`titanic3`)

The source data has moved a few times:

- TODO

The latest incarnation can be found here:

- https://hbiostat.org/data/
- https://hbiostat.org/data/repo/titanic
- https://hbiostat.org/data/repo/titanic3.csv

and has been replicated countless times including the Kaggle version.


Is it possible to find actual date of birth for each passanger? These data sources only have age (it seems).

These data sets were downloaded from https://www.kaggle.com/c/titanic/data

#### DuckDB

In [4]:
sql_get_data = f"CREATE OR REPLACE TABLE titanic AS SELECT * FROM read_csv('{DATA_URL}')"

In [5]:
sql_get_data

"CREATE OR REPLACE TABLE titanic AS SELECT * FROM read_csv('https://hbiostat.org/data/repo/titanic3.csv')"

In [6]:
conn = duckdb.connect()
conn.sql(sql_get_data)
duck_df = conn.sql("SELECT * FROM titanic").fetchdf()

In [11]:
info_df = conn.sql("PRAGMA show_tables_expanded;").df()

In [15]:
info_df["column_names"].to_list()

[array(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch',
        'ticket', 'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
       dtype=object)]

In [16]:
info_df["column_types"].to_list()

[array(['BIGINT', 'BIGINT', 'VARCHAR', 'VARCHAR', 'DOUBLE', 'BIGINT',
        'BIGINT', 'VARCHAR', 'DOUBLE', 'VARCHAR', 'VARCHAR', 'VARCHAR',
        'BIGINT', 'VARCHAR'], dtype=object)]

In [7]:
duck_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


#### Pandas (comparison)

In [ ]:
pd_df = pd.read_csv(DATA_URL)

In [ ]:
duck_df.equals(pd_df)

### EDA with `dataprep`

https://dataprep.ai


In [ ]:
data

In [ ]:
DATA_DIR = Path.cwd().parent / "data"
TRAIN_CSV = DATA_DIR / "train.csv"
TEST_CSV = DATA_DIR / "test.csv"

In [ ]:
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

In [ ]:
train_df.describe()

In [ ]:

fig = px.histogram(train_df, x='Age', nbins=50)
fig.show()


In [ ]:
def clean_and_extract_names(name):
    """
    Cleans up and extracts components from the 'Name' column of a DataFrame.
    
    Parameters:
    - name (str): The full name string from the DataFrame.
    
    Returns:
    - dict: A dictionary containing the title, surname, first name, and other names.
    """
    # Initialize default values for name components
    name_parts = {'title': '', 'surname': '', 'first_name': '', 'other_names': ''}
    
    try:
        # Clean name by replacing non-breaking spaces and trimming
        clean_name = name.replace(u'\xa0', ' ').strip()
        
        # Extract name parts using regular expression
        match = re.match(r'(?P<surname>[\w\s]+),\s?(?P<title>[\w\s]+)\.\s?(?P<rest>[\w\s\(\)]+)', clean_name)
        if match:
            name_parts['surname'] = match.group('surname')
            name_parts['title'] = match.group('title')
            rest = match.group('rest').split()
            
            # Assuming the first name is always the first word in the remaining string
            name_parts['first_name'] = rest[0]
            
            # The rest of the names are considered as other names
            if len(rest) > 1:
                name_parts['other_names'] = ' '.join(rest[1:])
                
            # Special handling for names within parentheses (typically for married women)
            paren_name_match = re.search(r'\(([\w\s]+)\)', clean_name)
            if paren_name_match:
                # Overwrite first name and clear other names if a parenthetical name is found
                name_parts['first_name'] = paren_name_match.group(1).split()[0]
                name_parts['other_names'] = ' '.join(paren_name_match.group(1).split()[1:])
                
    except Exception as e:
        print(f"Error processing name: {name}. Error: {e}")
    
    return name_parts

In [ ]:
train_df[['Title', 'Surname', 'Firstname', 'Othernames']] = train_df['Name'].apply(lambda x: pd.Series(clean_and_extract_names(x)))

In [ ]:
train_df

In [ ]:
train_df['Name'] = train_df['Name'].apply(lambda x : x.replace(u'\xa0', ' '))

In [ ]:
def extract_names(name_string, return_type):
    
    first_name = 'XXX NO FIRSTNAME XXX'
    other_names = ''
    
    name_str = name_string.split()
    if name_str[0][-1] == ',':
        surname = name_str[0][0:len(name_str[0])-1]
        title = name_str[1][0:len(name_str[1])-1]
        my_names = name_str[2:len(name_str)]
    else:
        surname = name_str[0] + ' ' + name_str[1][0:len(name_str[0])-1]
        title = name_str[2][0:len(name_str[2])-1]
        my_names = name_str[3:len(name_str)]

    first_name = my_names[0]
    other_names = ''
    for name in my_names[1:len(my_names)-1]:
        other_names = other_names + name + ' '
    other_names = other_names.rstrip()       
        
    try:
        if my_names[len(my_names)-1][-1] == ')':          # extract female/wife name
            other_names = ''
            for name in my_names:
                other_names = other_names + name + ' '
                if name[0] == '(':
                    other_names = ''
                    first_name = name[1:len(name)]
            other_names = other_names.rstrip()
            other_names = other_names[0:len(other_names)-1]             
    except:
        pass
    
    if return_type == 1:
        return title
    elif return_type == 2:
        return surname
    elif return_type == 3:
        return first_name
    else:
        return other_names

In [ ]:
train_df.info()

In [ ]:
test_df.info()

Merge the Kaggle train and test datasets into one (for comparison to other Titanic data sets/sources)

In [ ]:
df = pd.merge(train_df, test_df, how='outer')

In [ ]:
df.info()

So in the combined dataset of 1309 (= 891 train + 418 test) passengers, there are 1046 non-null age values.

Let's extract the components of the 'Name' field into title, surname, first name and other names.

In [ ]:
df['TitleOLd'] = df['Name'].apply(lambda x : extract_names(x, 1))
df['SurnameOld'] = df['Name'].apply(lambda x : extract_names(x, 2))
df['FirstnameOld'] = df['Name'].apply(lambda x : extract_names(x, 3))
df['OthernamesOld'] = df['Name'].apply(lambda x : extract_names(x, 4))

In [ ]:
df_sort_age = df.sort_values(by=['Age', 'Name'])
df_sort_age.reset_index(inplace=True)

Now put the other dataset(s) age data on this plot too.

In [ ]:
df_wikipedia = pd.read_html('https://en.m.wikipedia.org/wiki/Passengers_of_the_RMS_Titanic',header=0)

In [ ]:
facts_tables = pd.read_html('http://www.titanicfacts.net/titanic-passenger-list.html',header=0)

In [ ]:
facts_tables[0].info()

In [ ]:
facts_tables[1].info()

In [ ]:
facts_tables[2].info()

So the "Titanic Facts" has the data spread across 3 tables with 324 (1st class), 284 (2nd class) and 709 (3rd class) passengers (1317 total) respectively. There is age data for all 1317 passengers. Contrast this with 1309 passengers in the Kaggle dataset [which does not claim to be complete -- in fact, somewhat disappointingly, there does not appear to be a reference for the data] with only 1046 age values.

In [ ]:
facts = pd.merge(pd.merge(facts_tables[0], facts_tables[1], how='outer'), facts_tables[2], how='outer')

In [ ]:
facts.to_csv('facts.csv')

In [ ]:
facts.info()

In [ ]:
# Also could go heavy-handed with:
# pd.to_numeric(df['y'], errors='coerce')

def convert_age(age):
    try:
        return float(age)
    except:
        try:
            return float(age[0:len(age)-1]) / 12.0
        except:
            return np.nan

In [ ]:
facts['Age'] = facts['Age'].apply(lambda age : convert_age(age))

Need to convert age from string to float

In [ ]:
facts_sort_age = facts.sort_values(by=['Age', 'Surname'])
facts_sort_age.reset_index(inplace=True)

In [ ]:
len(facts_sort_age['Age'])

In [ ]:
#sns.plt.plot(facts_sort_age['Age'])
#sns.plt.plot(df_sort_age['Age'], 'g')

len(df_sort_age['Age'])

In [ ]:
df_sort_age.head()

In [ ]:
facts_sort_age.head()

In [ ]:
facts_sort_age.info()
print()
df_sort_age.info()

In [ ]:
facts_sort_age['Name'] = facts_sort_age['Surname'] + facts_sort_age['First Names']

Test the operation of the get_close_matches( ) method to see if we can merge the datasets based on the name fields.

In [ ]:
for i in range(0,5):
    name_str = dl.get_close_matches(df_sort_age['Name'].iloc[i], facts_sort_age['Name'])
    try:
        print(str(len(name_str)) + ': ' + name_str[0] + ' / ' + df_sort_age['Name'].iloc[i])
    except:
        print('No match' + ' / ' + df_sort_age['Name'].iloc[i])
              

In [ ]:
def match_name(name):
    try:
        return dl.get_close_matches(name, facts_sort_age['Name'])[0]
    except:
        return 'No name match'

df_sort_age['Name'] = df_sort_age['Name'].apply(lambda name: match_name(name))

In [ ]:
df_sort_age[df_sort_age['Name'] != 'No name match']

In [ ]:
df_sort_age['KaggleAge'] = df_sort_age['Age']

In [ ]:
merged = df_sort_age.merge(facts_sort_age, how='left', left_on='Name', right_on='Name')


In [ ]:
merged.info()

In [ ]:
merged['AgeDiff'] = merged['KaggleAge'] - merged['Age_y']

In [ ]:
merged['AgeDiff'] = merged['AgeDiff'].dropna()
merged['AgeDiff'].describe()

In [ ]:
merged.to_csv('merged.csv')

In [ ]:
merged['AgeDiffMoreEps'] = merged['AgeDiff'].apply(lambda agediff : abs(agediff) > 2)
#merged.corr()

In [ ]:
merged['AgeDiff'].describe()

In [ ]:
merged_df = merged[merged['AgeDiff'] != 0]
fig = px.histogram(merged_df, x='AgeDiff', nbins=75)
fig.show()

So for the most part, allowing for the NaN differences in the data sets they look pretty similar (other than the max age of 80). So let's look at some of the other top values.

In [ ]:
facts_sort_age['Age'].sort_values(na_position='first').tail(5)

In [ ]:
df_sort_age['Age'].sort_values(na_position='first').tail(5)

So it seems we have a 76 year old -- wonder if this age is also spurious?

In [ ]:
df_sort_age.iloc[1044]

Yes -- another one! Julia Florence Cavendish (who survived) was 25 at the time of the disaster. Her age at death was 76. c.f. https://www.encyclopedia-titanica.org/titanic-survivor/julia-florence-cavendish.html